# Neural Networks: A 2:45 Hour Deep Dive
This notebook covers perceptrons, feed-forward networks, backpropagation, and specialized architectures (CNNs, RNNs, GNNs) with practical implementation and visualizations.

## Perceptrons
A perceptron is the simplest form of a neural network. It's a binary classifier that maps its input `x` (a real-valued vector) to an output value `f(x)` (a single binary value).

## Feed-Forward Neural Networks
These are the most basic type of artificial neural network architecture.

## Backpropagation
The learning algorithm used for training the weights in FFNNs.

## Scikit-Learn MLP Example with Mixed Data Types

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
import pandas as pd

# Load Titanic dataset as an example
data = fetch_openml('titanic', version=1, as_frame=True)
df = data.frame.dropna(subset=['age', 'fare', 'embarked', 'sex'])

# Features and target
X = df[['pclass', 'sex', 'age', 'fare', 'embarked']]
y = df['survived']

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

# Preprocessing
numeric_features = ['age', 'fare']
categorical_features = ['pclass', 'sex', 'embarked']

preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(), categorical_features)
])

# Pipeline
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', MLPClassifier( hidden_layer_sizes=(10,), max_iter=300, random_state=1))
])

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.92      0.85       155
           1       0.84      0.66      0.74       106

    accuracy                           0.81       261
   macro avg       0.82      0.79      0.80       261
weighted avg       0.82      0.81      0.81       261



In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load Titanic dataset
data = fetch_openml('titanic', version=1, as_frame=True)
df = data.frame.dropna(subset=['age', 'fare', 'embarked', 'sex'])

# Define features and target
X = df[['pclass', 'sex', 'age', 'fare', 'embarked']]
y = df['survived']

# Encode target variable
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, stratify=y_encoded, random_state=42)

# Define feature types
string_cats = ['sex', 'embarked']
int_cats = ['pclass']
numeric_features = ['age', 'fare']

# Create input layers
inputs = {}
for col in string_cats:
    inputs[col] = tf.keras.Input(shape=(1,), name=col, dtype=tf.string)

for col in int_cats + numeric_features:
    inputs[col] = tf.keras.Input(shape=(1,), name=col, dtype=tf.float32)

# Normalization for numeric features
norm = layers.Normalization()
norm.adapt(np.array(X_train[numeric_features]))

numeric_inputs = [inputs[col] for col in numeric_features]
concatinated_layers = tf.keras.layers.Concatenate()(numeric_inputs)
numeric_encoded = norm(concatinated_layers)

# Integer categorical encoding
int_encoded = []
for col in int_cats:
    vocab = sorted(X_train[col].unique())
    vocab = [int(v) for v in vocab]
    lookup = layers.IntegerLookup(vocabulary=vocab, output_mode="one_hot")
    int_encoded.append(lookup(inputs[col]))

# String categorical encoding
string_encoded = []
for col in string_cats:
    vocab = sorted(X_train[col].unique())
    lookup = layers.StringLookup(vocabulary=vocab, output_mode="one_hot")
    string_encoded.append(lookup(inputs[col]))

# Combine all features
all_features = layers.Concatenate()( [numeric_encoded] + int_encoded + string_encoded )

# Build the model
x = layers.Dense(128, activation='relu')(all_features)
x = layers.Dense(64, activation='relu')(x)
output = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Prepare input data as dict
train_dict = {name: np.array(X_train[name]) for name in X.columns}
test_dict = {name: np.array(X_test[name]) for name in X.columns}

# Train and evaluate
model.



Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - accuracy: 0.5364 - loss: 0.6678 - val_accuracy: 0.7962 - val_loss: 0.5442
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 73ms/step - accuracy: 0.7789 - loss: 0.5276 - val_accuracy: 0.8153 - val_loss: 0.4630
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7758 - loss: 0.4685 - val_accuracy: 0.8153 - val_loss: 0.4331
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7763 - loss: 0.4819 - val_accuracy: 0.8025 - val_loss: 0.4376
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8099 - loss: 0.4124 - val_accuracy: 0.8089 - val_loss: 0.4170
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7899 - loss: 0.4489 - val_accuracy: 0.7898 - val_loss: 0.4304
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7925 - loss: 0.4479 - val_accuracy: 0.8089 - val_loss: 0.4179
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7932 - loss: 0.4433 - val_accuracy: 0.8089 - v

[0.45637714862823486, 0.8160919547080994]

In [ ]:
print(all_features)

<KerasTensor shape=(None, 13), dtype=float32, sparse=False, name=keras_tensor_15>


In short, CNN is good at image processing, RNN is good at sequential data processing and GAN is good at generative tasks.

![Commonly-used-deep-learning-models-CNN-RNN-and-GAN.ppm](...)